In [1]:
import os
import sys
sys.path.append(".." + os.sep + "..")

import torch
import syft as sy

from src.future import PartitionedDataset, VerticalDataset

Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/home/tom/anaconda3/envs/pyvertical-dev/lib/python3.7/site-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.15.3.so'


In [2]:
hook = sy.TorchHook(torch)

We will now turn this dataset into a PartitionedDataset. PartitionedDatsets can hold data, targets or both.

In [3]:
data = torch.tensor([1.0, 2.0, 3.0]).tag("#toy").describe("Toy input data.")
targets = torch.tensor([0, 1, 1]).tag("#toy").describe("Toy data labels.")
dataset = PartitionedDataset(data, targets)

Just one dataset isn't very exciting - the data is not vertically partitioned! PartitionedDatasets come with a helper method to vertically partition a dataset.
We will move the data onto virtual workers.

In [4]:
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)

In [5]:
vertical_data = dataset.vertically_federate((alice, bob))

In [6]:
type(vertical_data)

src.future.dataset.VerticalDataset

This new dataset is a VerticalDataset. This is similar to syft's FederatedDataset - it holds a list of vertically partitioned dataset assigned to different workers.

In [7]:
alice_results = alice.search(["#toy"])
for res in alice_results:
    print(res.description)

Toy input data.


In [8]:
bob_results = bob.search(["#toy"])
for res in bob_results:
    print(res.description)

Toy data labels.


You can see that Alice has the data and Bob has the labels.

In [9]:
vertical_data.workers

['alice', 'bob']

You can collect a dataset from its remote.

In [10]:
alices_dataset = vertical_data.get_dataset("alice")

In [11]:
alices_dataset

PartitionedDataset
	Data: tensor([1., 2., 3.])

In [12]:
vertical_data.workers

['bob']

After which the VerticalDataset only contains Bob's labels.